# Getting Stock Data
First, lets import all the stuff we'll need to get pricing data

In [1]:
import numpy as np
import pandas as pd
import dill
import quandl
from sqlalchemy import create_engine, inspect
from tqdm import tqdm_notebook

Second things second, lets get our set of ticker symbols out of the pickle jar, and into a sorted list

In [2]:
tickSymbs = sorted(dill.load(open('setOfTickerSymbols.pkl', 'r')))

Now lets open my Quandl API key from its hidden and ignored file (so nobody scrapes it off this github repo)

In [3]:
quandl_key_handle = open("quandl.apikey", "r")
quandl.ApiConfig.api_key = quandl_key_handle.read()
quandl_key_handle.close()

#### Now lets download some pricing data!
...straight to a sqlite database now, no more dicts!

In [14]:
engine = create_engine('sqlite:///capstone.db')

In [15]:
dlFails = []
dlWins = []
for ticker in tqdm_notebook(tickSymbs):
    try:
        #tickerDF = quandl.get("WIKI/%s" % ticker)
        quandl.get("WIKI/%s" % ticker).to_sql(ticker, engine, if_exists = 'replace')
    except:
        dlFails.append(ticker)
    else:
        dlWins.append(ticker)

So I was able to download about 1/3 of my tickers of interest from Quandl. Lets make sure that the API request and `pandas.to_sql()` method actually did what I intended.

In [17]:
inspector = inspect(engine)
print " ".join(inspector.get_table_names())

ABBV ACOR ACRX ADMS AERI AGEN AGIO AGN AGTC AKAO AKBA ALDR ALKS ALNY ALXN AMAG AMGN ANIK ARNA ARQL ARRY ATNM ATRS AVEO BCRX BIIB BLUE BMRN BMY BPTH BTX CELG CEMP CLDX CORT CTIC CYTK CYTR DVAX EBIO EGLT EPZM ESPR EXEL FCSC FLXN FOLD GALE GALT GERN GILD GTXI HALO HRTX HZNP ICPT IDRA IMGN INCY IRWD JNJ KERX KPTI LGND LLY LXRX MACK MDCO MEIP MRK MRTX MYL NBIX NKTR NLNK NVAX OHRP OPHT OPK PBYI PFE PGNX PRTA PTCT PTLA RARE REGN RGLS RIGL RVNC SGEN SGMO SGYP SPPI SRPT STML TBPH TGTX TSRO TTPH TXMD UTHR VICL VNDA VRTX VRX VSAR VTL XLRN XNCR ZGNX


So now we have a sqlite DB with _some_ of the stock histories we wanted. Lets start feature extractions.
If needed, I'll buy premium DB access from Quandl and get the whole NYSE. 